In [1]:
import torch
import torch.nn as nn
import torch.optim as optim



In [3]:
#cancer
import pandas as pd
import numpy as np

data = pd.read_csv('./data/train.csv')

data.head(1)

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [7]:
# 2번째 컬럼부터 유전자 정보 선택
df = data.iloc[:, 2:].copy()

# 유전자 부위를 숫자로 변환 (매핑 저장)
gene_map = {gene: idx for idx, gene in enumerate(df.columns)}



In [8]:
# "WT"가 아닌 값들을 0/1 변이 행렬로 변환
mutation_matrix = df.applymap(lambda x: 1 if x != "WT" else 0)

print(mutation_matrix)

# 공존 행렬 생성 (변이 행렬의 전치 행렬과 원본 행렬의 내적)
co_occurrence_matrix = mutation_matrix.T.dot(mutation_matrix)

# 자기 자신과의 공존(대각선 요소)은 0으로 설정
np.fill_diagonal(co_occurrence_matrix.to_numpy(), 0)


print(co_occurrence_matrix)


      A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  ABCA4  ABCA5  ...  \
0       0     0      0      0     0      0      0      0      0      0  ...   
1       0     0      0      0     0      0      0      0      0      0  ...   
2       1     0      0      0     0      0      0      0      0      0  ...   
3       0     0      0      0     0      0      0      0      0      0  ...   
4       0     0      0      0     0      0      0      0      0      0  ...   
...   ...   ...    ...    ...   ...    ...    ...    ...    ...    ...  ...   
6196    0     0      0      0     0      0      0      0      0      0  ...   
6197    0     0      0      0     0      0      0      0      0      0  ...   
6198    0     0      0      0     0      0      0      0      0      0  ...   
6199    0     0      0      0     0      0      0      0      0      0  ...   
6200    0     0      0      0     0      0      0      0      0      0  ...   

      ZNF292  ZNF365  ZNF639  ZNF707  ZNFX1  ZNRF4 

In [9]:
occurrence_matrix = co_occurrence_matrix.to_numpy()

In [10]:
import numpy as np
from sklearn.preprocessing import normalize

# 3. SVD
U, S, Vt = np.linalg.svd(occurrence_matrix)

# 4. Embedding 선택
k = 150
gene_embeddings = U[:, :k] * np.sqrt(S[:k])  # shape: [num_genes, k]

# 5. normalize (선택)
gene_embeddings = normalize(gene_embeddings, axis=1)

In [18]:
import os

In [20]:
np.save(os.path.join("data", "gene_embedSVD.npy"), gene_embeddings)